# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [103]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [104]:
city_file = "../WeatherPy/Output/city_data_2020-04-12.csv"
city_df = pd.read_csv(city_file)
city_df.dropna() # Should be none as NA Values were dropped during the creation of the file
city_df.head()

,City,Date_Updated,Country,Latitude,Longitude,Temperature_Max,Humidity,Cloudiness,Wind_Speed
0,basco,2020-04-12,PH,20.45,121.97,69.46,71,87,22.79
1,atbasar,2020-04-12,KZ,51.80,68.33,39.90,64,4,7.63
2,souillac,2020-04-12,MU,-20.52,57.52,78.01,83,75,13.87
3,kruisfontein,2020-04-12,ZA,-34.00,24.73,54.12,80,3,7.09
4,batemans bay,2020-04-12,AU,-35.72,150.18,57.00,100,100,2.66


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [105]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_df[["Latitude", "Longitude"]]

# Populate Humidity and store as float
Humidity = city_df["Humidity"].astype(float)

In [106]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [107]:
city_df = city_df.loc[(city_df['Temperature_Max']  > 80)
               & (city_df['Temperature_Max']  < 90)
               & (city_df['Cloudiness']  < 5)
                          ]
city_df

,City,Date_Updated,Country,Latitude,Longitude,Temperature_Max,Humidity,Cloudiness,Wind_Speed
42,hilo,2020-04-12,US,19.73,-155.09,82.40,65,1,11.41
120,monywa,2020-04-12,MM,22.12,95.13,80.10,35,0,5.21
303,biak,2020-04-12,ID,-0.91,122.88,81.23,67,3,1.41
426,dogondoutchi,2020-04-12,NE,13.64,4.03,87.98,12,0,12.41
463,diu,2020-04-12,IN,20.71,70.98,81.16,45,0,12.86
488,hervey bay,2020-04-12,AU,-25.30,152.85,80.01,68,3,10.29
504,barra patuca,2020-04-12,HN,15.80,-84.28,82.67,76,0,21.47
538,san lorenzo,2020-04-12,PY,-25.33,-57.53,80.60,34,0,4.70
543,charters towers,2020-04-12,AU,-20.10,146.27,81.18,55,0,10.40


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [108]:
#Create dataframe name hotel_df with the ideal locations created above
Hotel_df = city_df
Hotel_df['Hotel_Name'] = ""
Hotel_df

,City,Date_Updated,Country,Latitude,Longitude,Temperature_Max,Humidity,Cloudiness,Wind_Speed,Hotel_Name
42,hilo,2020-04-12,US,19.73,-155.09,82.40,65,1,11.41,
120,monywa,2020-04-12,MM,22.12,95.13,80.10,35,0,5.21,
303,biak,2020-04-12,ID,-0.91,122.88,81.23,67,3,1.41,
426,dogondoutchi,2020-04-12,NE,13.64,4.03,87.98,12,0,12.41,
463,diu,2020-04-12,IN,20.71,70.98,81.16,45,0,12.86,
488,hervey bay,2020-04-12,AU,-25.30,152.85,80.01,68,3,10.29,
504,barra patuca,2020-04-12,HN,15.80,-84.28,82.67,76,0,21.47,
538,san lorenzo,2020-04-12,PY,-25.33,-57.53,80.60,34,0,4.70,
543,charters towers,2020-04-12,AU,-20.10,146.27,81.18,55,0,10.40,


In [109]:
# find the closest restaurant of each type to coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_distance = 5000 

params = {
    "type": "lodging",
    "distance": hotel_distance,
    "rankby": "distance",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in Hotel_df.iterrows():

    City = row['City']

    # update location keyword based off row being read from csv and adding to params dict
    Latitude = row['Latitude']
    Longitude = row['Longitude']
    params['location'] = f"{Latitude},{Longitude}"

    # assemble url and make API request
    print(f"Requesting Requests for Index {index} --City:  {City}")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel to {City} is {results[0]['name']}.")
        Hotel_df.loc[index, 'Hotel_Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        print(f"{results}")
        
    print("------------")

Requesting Requests for Index 42 --City:  hilo
Closest hotel to hilo is Dolphin Bay Hotel.
------------
Requesting Requests for Index 120 --City:  monywa
Closest hotel to monywa is Monywa Hotel.
------------
Requesting Requests for Index 303 --City:  biak
Closest hotel to biak is Kost Ariz.
------------
Requesting Requests for Index 426 --City:  dogondoutchi
Closest hotel to dogondoutchi is Hôtel Alheri Maison Alheri.
------------
Requesting Requests for Index 463 --City:  diu
Closest hotel to diu is Hotel Hemal Garden.
------------
Requesting Requests for Index 488 --City:  hervey bay
Closest hotel to hervey bay is Discovery Parks - Boat Harbour Drive, Hervey Bay.
------------
Requesting Requests for Index 504 --City:  barra patuca
Missing field/result... skipping.
[]
------------
Requesting Requests for Index 538 --City:  san lorenzo
Closest hotel to san lorenzo is EDIFICIO ORTELLADO.
------------
Requesting Requests for Index 543 --City:  charters towers
Closest hotel to charters to

In [110]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in Hotel_df.iterrows()]
locations = Hotel_df[["Latitude", "Longitude"]]

In [111]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))